In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-
"""
Created on Sat Nov 17 14:11:56 2018

@author: Eric
"""

import requests
import csv
import json
from statistics import *
from collections import Counter

In [ ]:
#prep the outputs
obama={'timeNV':{},
'timeSNC':{},
'timeV':{},
'timeU':{},
'codes':{},
'count':{}
}
trump={'timeNV':{},
'timeSNC':{},
'timeV':{},
'timeU':{},
'codes':{},
'count':{}
}

In [ ]:
# The main script
# Loop through each EPA region and get the ECHO results for compliance information
x=1
while x < 11:
    region=str(x)
    if len(region)<2:
        region = "0"+region
    print(region) # Print out which region we're currently working on
    
    url="https://ofmpub.epa.gov/echo/echo_rest_services.get_facility_info?output=json&p_reg="+region+"&p_act=Y&p_maj=Y&p_med=ALL"
    contents = requests.get(url).content.decode()
    obj = json.loads(contents) # Get the results from ECHO and turn them into something Python can work with
    
    facilities = obj['Results']['Facilities'] # Grab the information on the facilities
    region="R"+region # Name the current region
    
    # Set up the data a bit more
    for key in obama:
        obama[key][region]=[]
        trump[key][region]=[]
    obama['count'][region]=0
    trump['count'][region]=0
    
    # For each facility in the EPA region...
    for facility in facilities:
         compliance = facility['Fac3yrComplianceHistory']  # Get its compliance history for the past 12 quarters in the form of "123456789012" #V = violation U = unknown S =sign. violation _ = none
         if (compliance[0:3] == "UUU"): #if the first three quarters in the Obama era were Unknown status, skip because they may not have been permitted at all then, only later.
             pass
         else:
             #print(compliance)
                
             t = compliance[8:12] # For this analysis, we measure the Trump era as the most recent 4 quarters
             o = compliance[0:4] # The Obama era is the first 4 quarters of the past 12 (as of Nov 2018)
                
             # Measure the percent of time (number of quarters) the facility spent in each compliance category
             trump['timeNV'][region].append(t.count("_")/4)# No violation
             trump['timeSNC'][region].append(t.count("S")/4) # Significant non compliance
             trump['timeV'][region].append(t.count("V")/4) # Violation
             trump['timeU'][region].append(t.count("U")/4) # Unknown status
             trump['codes'][region].append(compliance[10]) #Quarter 11 For purposes of measuring the percent of facilities in a compliance category at any given time (rather than the percent of time each one spent in each category) we use Q10 for Trump, Q2 for Obama (summer 2016/2018)
             trump['count'][region]+=1 
             
             # Measure the percent of time (number of quarters) the facility spent in each compliance category
             obama['timeNV'][region].append(o.count("_")/4)
             obama['timeSNC'][region].append(o.count("S")/4)
             obama['timeV'][region].append(o.count("V")/4)
             obama['timeU'][region].append(o.count("U")/4)
             obama['codes'][region].append(compliance[2]) #Quarter 3
             obama['count'][region]+=1
        # Go to the next facility...
    x+=1 # Go to the next EPA region

In [ ]:
# OUTPUT: percent of facilities in each compliance category (Quarter 3 vs Quarter 11)
print("OBAMA")
for region in obama['codes']:
    total=Counter(obama['codes'][region])
    for key in total:
        pct=100*(total[key]/len(obama['codes'][region]))
        print(region +" "+ key +" "+ str(pct))
print("TRUMP")
for region in trump['codes']:
    total=Counter(trump['codes'][region])
    for key in total:
        pct=100*(total[key]/len(trump['codes'][region]))
        print(region +" "+ key +" "+ str(pct))

In [ ]:
#OUTPUT: for each region, print out the average amount of time facilities spent in each compliance category (as %)
for key in obama:
    if (key == 'count') or (key=='codes'):
        pass
    else:
        for region in obama[key]:
            m = mean(obama[key][region])*100
            print ("OBAMA "+region+" "+str(key)+" "+str(m))
for key in trump:
    if (key == 'count') or (key=='codes'):
        pass
    else:
        for region in trump[key]:
            m = mean(trump[key][region])*100
            print ("TRUMP "+region+" "+str(key)+" "+str(m))

In [ ]:
# Calculate mean percent of time facilities spent in each compliance status nationally rather than by region          
ntl={"obama": {}, "trump": {}}
for key in obama:
    if (key != 'count') and (key != 'codes'):
        ntl['obama'][key]=[]
        for region in obama[key]:
            for val in obama[key][region]:
                ntl['obama'][key].append(val)
for key in trump:
    if (key != 'count') and (key != 'codes'): 
        ntl['trump'][key]=[]
        for region in trump[key]:
            for value in trump[key][region]:
                ntl['trump'][key].append(value)
                
# Print results from above ^^^              
for key in ntl['obama']:
    m=mean(ntl['obama'][key])*100
    print ("OBAMA "+str(key)+" "+str(m))
    
for key in ntl['trump']:
    m=mean(ntl['trump'][key])*100
    print ("trump "+str(key)+" "+str(m))